In [0]:
!pip install databricks-sdk==0.52.0 mlflow==2.22.0 databricks-agents==0.22.0 openai uv --quiet
dbutils.library.restartPython()

In [0]:
from agent import agent 
#from agent import agent
agent.predict({"messages": [{"role": "user", "content": "Are there any long term housing options available?"}]})

In [0]:
from agent import agent 
#from agent import agent
agent.predict({"messages": [{"role": "user", "content": "How to get started with reunifying process?"}]})

In [0]:
for event in agent.predict_stream(
    {"messages": [{"role": "user", "content": "Can you tell me a joke?"}]}
):
    print(event, "-----------\n")

In [0]:
# Determine Databricks resources to specify for automatic auth passthrough at deployment time
import mlflow
from mlflow.models.auth_policy import SystemAuthPolicy, UserAuthPolicy, AuthPolicy

from mlflow.models.resources import (
  DatabricksVectorSearchIndex,
  DatabricksServingEndpoint,
  DatabricksSQLWarehouse,
  DatabricksFunction,
  DatabricksGenieSpace,
  DatabricksTable,
  DatabricksUCConnection
)

resources = [DatabricksServingEndpoint(endpoint_name='demo2')]


# Specify resources here for system authentication
system_auth_policy = SystemAuthPolicy(resources=resources)

# Specify the minimal set of API scopes needed for on-behalf-of-user authentication
# When deployed, the agent can access Databricks resources and APIs
# on behalf of the end user, but only via REST APIs that are covered by the list of
# scopes below

user_auth_policy = UserAuthPolicy(
    api_scopes=[
        "dashboards.genie"
    ]
)

input_example = {
    "messages": [
        {
            "role": "user",
            "content": "Tell me a joke"
        }
    ]
}

with mlflow.start_run():
    logged_agent_info = mlflow.pyfunc.log_model(
        artifact_path="agent",
        python_model="agent.py",
        code_paths=["./src"],
        input_example=input_example,
        pip_requirements=[
            "mlflow==2.22.0",
            "databricks-sdk==0.52.0",
            "openai==1.69.0",
            "pydantic",
            # "llama-index==0.12.41",
            # "llama-index-llms-databricks==0.3.2",
            # "llama-index-tools-mcp==0.2.5",
            # "python-dotenv==1.1.0",
            # "async"
        ],
        auth_policy=AuthPolicy(
            system_auth_policy=system_auth_policy,
            user_auth_policy=user_auth_policy
        )
    )


In [0]:
# import asyncio
# from llama_index.llms.databricks import Databricks
# from databricks.sdk import WorkspaceClient
# import mlflow
# from dotenv import load_dotenv
# from llama_index.tools.mcp import BasicMCPClient, McpToolSpec
# import os
# from llama_index.core.agent.workflow import ReActAgent

# class NimbleResearchAgent:
#     def __init__(self, nimble_api_key, databricks_model="demo2"):
#         self.nimble_api_key = nimble_api_key
#         self.databricks_model = databricks_model
#         self.agent = None
#         self.tools = None
        
#     async def setup(self):
#         """Set up the agent with Nimble tools and Databricks LLM."""
#         # Set up Nimble tools
#         self.tools = await self._setup_nimble_tools()
        
#         # Set up Databricks LLM
#         w = WorkspaceClient()
#         tmp_token = w.tokens.create(comment="for model serving", lifetime_seconds=1200).token_value
        
#         llm = Databricks(
#             model=self.databricks_model,
#             api_key=tmp_token,
#             api_base=f"{w.config.host}/serving-endpoints/"
#         )
        
#         # Create the agent
#         self.agent = ReActAgent(
#             tools=self.tools,
#             llm=llm,
#             system_prompt="""You are a helpful research assistant with access to web scraping and data collection tools. 
#             Always explain your answer in the final output. Tell the user which tools you used and how you found the information.""",
#         )
        
#         return self
        
#     async def _setup_nimble_tools(self):
#         """Set up and return the Nimble tools."""
#         mcp_client = BasicMCPClient(
#             "https://mcp.nimbleway.com/sse",
#             headers={"Authorization": f"Bearer {self.nimble_api_key}"}
#         )
        
#         mcp_tool_spec = McpToolSpec(
#             client=mcp_client,
#             # Optional: filter to specific tools
#             # allowed_tools=["nimble_web_search", "nimble_google_maps_search"]
#         )
        
#         tools = await mcp_tool_spec.to_tool_list_async()
        
#         print(f"Loaded {len(tools)} Nimble tools:")
#         for tool in tools:
#             print(f"- {tool.metadata.name}: {tool.metadata.description}")
        
#         return tools
    
#     async def run_query(self, query):
#         """
#         Run a query through the agent.
        
#         Args:
#             query: The query string to process
            
#         Returns:
#             The agent's response
#         """
#         if not self.agent:
#             raise ValueError("Agent not initialized. Call setup() first.")
        
#         return await self.agent.run(query)



In [0]:
mlflow.set_registry_uri("databricks-uc")

# TODO: define the catalog, schema, and model name for your UC model
catalog = "workspace"
schema = "default"
model_name = "refu_genie"
UC_MODEL_NAME = f"{catalog}.{schema}.{model_name}"

# register the model to UC
uc_registered_model_info = mlflow.register_model(
    model_uri=logged_agent_info.model_uri, name=UC_MODEL_NAME
)

In [0]:
from databricks import agents
agents.deploy(UC_MODEL_NAME, uc_registered_model_info.version, tags = {"endpointSource": "playground"})